In [ ]:
pip install openai

In [ ]:
import pandas as pd
from model2vec import StaticModel
from sklearn.cluster import KMeans
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern
from scipy.stats import multivariate_normal as mvn
#import json
#import boto3
#import re
#from scipy.stats import spearmanr

In [ ]:
train_df = pd.read_csv('~/environment/projects/no_repo/dspy/train.csv', encoding='iso-8859-1')
product_desc_df = pd.read_csv('~/environment/projects/no_repo/dspy/product_descriptions.csv', encoding='iso-8859-1')

In [ ]:
train_df.value_counts('product_uid').head()

In [ ]:
description = product_desc_df[product_desc_df['product_uid']==102456]['product_description'].iloc[0]
product_name = train_df[train_df['product_uid']==102456]['product_title'].iloc[0]
#.merge(product_desc_df, on='product_uid')

In [ ]:
model = StaticModel.from_pretrained("minishlab/potion-base-8M")
short_model = StaticModel.from_pretrained("minishlab/potion-base-2M")

In [ ]:
df = train_df.merge(product_desc_df, on='product_uid')
desc_embed = short_model.encode([t for t in df['product_description']])
search_embeddings = short_model.encode([t for t in df['search_term']])

model = StaticModel.from_pretrained("minishlab/potion-base-2M")
unique_search_embeddings = model.encode([t for t in df['search_term'].unique()])


In [ ]:
cluster = KMeans(n_clusters=768)
cluster.fit(unique_search_embeddings)
cluster_id = cluster.predict(search_embeddings)

In [ ]:
first_cluster_search = list(set([s for x, s in zip(cluster_id, df['search_term']) if x == 0]))
first_cluster_search

In [ ]:
from scipy.stats import ecdf
from scipy.stats import norm

transf = ecdf(train_df['relevance'])

# can't use 1 or 0 in norm.ppf. They translate to inf, -inf
normal_relevance = [norm.ppf(x*.999) + .0005 for x in transf.cdf.evaluate(train_df['relevance'])]

In [ ]:
first_cluster_desc = [s for x, s in zip(cluster_id, df['product_description']) if x == 0]
first_cluster_desc_embed = [de for x, de in zip(cluster_id, desc_embed) if x == 0]
first_cluster_product_id = [s for x, s in zip(cluster_id, df['product_uid']) if x == 0]
first_cluster_relevance = [r for x, r in zip(cluster_id, normal_relevance) if x == 0]

print(len(first_cluster_product_id), len(set(first_cluster_product_id)))
dedup_df = pd.DataFrame({'embedding': [str(x) for x in first_cluster_desc_embed],
                         'relevance': first_cluster_relevance}).drop_duplicates()


In [ ]:
from math import comb

n_recommendations = 4
comb(len(first_cluster_desc), n_recommendations)


In [ ]:
gpr = GaussianProcessRegressor(kernel= Matern() + WhiteKernel(), copy_X_train=False)
gpr.fit(first_cluster_desc_embed, first_cluster_relevance)

recommendations = pd.DataFrame({'embedding': first_cluster_desc_embed,
                                'relevance': first_cluster_relevance}).\
                  sort_values('relevance', ascending=False).\
                  head(n_recommendations)
top_rec_mu, top_rec_sig = gpr.predict([x for x in recommendations['embedding']], return_cov=True)
1-mvn.cdf([3]*n_recommendations, top_rec_mu, top_rec_sig)

In [ ]:
import qc
from importlib import reload
reload(qc)

In [ ]:
pip install git+http://github.com/quante_carlo.git

In [ ]:
from multiprocessing import Pool
import quante_carlo as qc
bo = qc.BEO(bo_batch_size=1000)

p = Pool(4)
results = p.map(bo.suggestion_combinations, [{'embeddings': first_cluster_desc_embed, 
                                              'scores': first_cluster_relevance}] * 4)
p.close()

In [ ]:
p.close()

In [ ]:
results